In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [2]:

chrome_options = Options()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--disable-gpu')
driver = webdriver.Chrome(executable_path=r'C:\Program Files (x86)\Google\chromedriver.exe', chrome_options=chrome_options)
url = 'https://www.scoregg.com/data/hero?tournamentID=191'
driver.get(url)
wait = WebDriverWait(driver,10)

## Get column names
table1=wait.until(EC.visibility_of_element_located((By.XPATH,'//*[contains(@class,"data-table fixed")]//tr')))
table2=wait.until(EC.visibility_of_element_located((By.XPATH,'//*[contains(@class,"data-table main")]//tr')))
data1 = [item.text for item in table1.find_elements_by_xpath(".//*[self::td or self::th]")]
data2 = [item.text for item in table2.find_elements_by_xpath(".//*[self::td or self::th]")]
col_Name=data1+data2
print("Column names is:",col_Name)

## finish clicking the Load More button
count=0
while True:
    try:
        item = wait.until(EC.visibility_of_element_located((By.XPATH,'//*[contains(@class,"table_page")]/img')))
        driver.execute_script("arguments[0].click();", item)
        count+=1
        print("Good click %d"%count,end='->')
    except Exception:break
        
## get all row elements one by one
row_hero=driver.find_elements_by_css_selector(".data-table.fixed >tbody >tr>td")
row_hero = [item.text for item in row_hero]
row_main=driver.find_elements_by_css_selector(".data-table.main >tbody >tr>td")
row_main = [item.text for item in row_main]



Column names is: ['排名', '英雄', '位置', '出场率', '被禁率', '胜率', 'KDA']
Good click 1->Good click 2->Good click 3->Good click 4->Good click 5->Good click 6->Good click 7->Good click 8->Good click 9->Good click 10->Good click 11->Good click 12->Good click 13->Good click 14->Good click 15->Good click 16->Good click 17->Good click 18->Good click 19->Good click 20->

In [3]:
len(row_hero)//2

156

In [4]:
len(row_main)//5

156

In [5]:
## Merge Rows
def mergeRows(row_hero,row_main):
    row_hero=[row_hero[i:i+2] for i in range(0,len(row_hero),2)]
    row_main=[row_main[i:i+5] for i in range(0,len(row_main),5)]
    assert(len(row_hero)==len(row_main))
    ans=[]
    for i in range(len(row_hero)):
        ans.append(row_hero[i]+row_main[i])
    return ans

row_ans=mergeRows(row_hero,row_main)
print(row_ans[0])
print(row_ans[1])

['1', '虚空之女', 'ADC', '54.5 % ，162 场', '40.7 %，121 场', '58.0 %，94 场', '4.4 KDA\n5.2 / 2.2 / 4.4']
['2', '牛头酋长', '辅助', '51.2 % ，152 场', '26.9 %，80 场', '51.3 %，78 场', '3.0 KDA\n0.8 / 3.1 / 8.6']


In [6]:
## Format to pd.DataFrame
LOL_Heros=pd.DataFrame(row_ans,columns=col_Name)
LOL_Heros

,排名,英雄,位置,出场率,被禁率,胜率,KDA
0,1,虚空之女,ADC,54.5 % ，162 场,40.7 %，121 场,58.0 %，94 场,4.4 KDA\n5.2 / 2.2 / 4.4
1,2,牛头酋长,辅助,51.2 % ，152 场,26.9 %，80 场,51.3 %，78 场,3.0 KDA\n0.8 / 3.1 / 8.6
2,3,迷失之牙,上单,41.1 % ，122 场,42.8 %，127 场,53.3 %，65 场,2.8 KDA\n2.0 / 2.6 / 5.3
3,4,兽灵行者,打野,40.4 % ，120 场,22.6 %，67 场,45.0 %，54 场,3.9 KDA\n2.4 / 2.2 / 6.2
4,5,发条魔灵,中单,37.0 % ，110 场,31.0 %，92 场,56.4 %，62 场,5.4 KDA\n3.0 / 1.9 / 7.2
...,...,...,...,...,...,...,...
151,152,复仇之矛,-,0.0 % ，0 场,4.1 %，2 场,0.0 %，0 场,0.0 KDA\n0.0 / 0.0 / 0.0
152,153,时间刺客,-,0.0 % ，0 场,1.1 %，2 场,0.0 %，0 场,0.0 KDA\n0.0 / 0.0 / 0.0
153,154,诡术妖姬,-,0.0 % ，0 场,8.3 %，1 场,0.0 %，0 场,0.0 KDA\n0.0 / 0.0 / 0.0
154,155,深海泰坦,-,0.0 % ，0 场,4.9 %，2 场,0.0 %，0 场,0.0 KDA\n0.0 / 0.0 / 0.0


In [7]:
LOL_Heros=LOL_Heros[LOL_Heros['位置']!='-']
LOL_Heros

,排名,英雄,位置,出场率,被禁率,胜率,KDA
0,1,虚空之女,ADC,54.5 % ，162 场,40.7 %，121 场,58.0 %，94 场,4.4 KDA\n5.2 / 2.2 / 4.4
1,2,牛头酋长,辅助,51.2 % ，152 场,26.9 %，80 场,51.3 %，78 场,3.0 KDA\n0.8 / 3.1 / 8.6
2,3,迷失之牙,上单,41.1 % ，122 场,42.8 %，127 场,53.3 %，65 场,2.8 KDA\n2.0 / 2.6 / 5.3
3,4,兽灵行者,打野,40.4 % ，120 场,22.6 %，67 场,45.0 %，54 场,3.9 KDA\n2.4 / 2.2 / 6.2
4,5,发条魔灵,中单,37.0 % ，110 场,31.0 %，92 场,56.4 %，62 场,5.4 KDA\n3.0 / 1.9 / 7.2
...,...,...,...,...,...,...,...
122,123,瓦洛兰之盾,辅助,0.3 % ，1 场,0.0 %，0 场,0.0 %，0 场,1.2 KDA\n0.0 / 5.0 / 6.0
123,124,暮光之眼,上单,0.3 % ，1 场,0.0 %，0 场,100.0 %，1 场,16.0 KDA\n3.0 / 1.0 / 13.0
124,125,不屈之枪,中单,0.3 % ，1 场,0.0 %，0 场,0.0 %，0 场,4.0 KDA\n3.0 / 3.0 / 9.0
125,126,虚空遁地兽,打野,0.3 % ，1 场,0.0 %，0 场,0.0 %，0 场,1.0 KDA\n4.0 / 6.0 / 2.0


In [8]:
LOL_Heros.to_csv('LOL_Heros.csv',index=None,encoding='utf-8-sig')

## Another way to go: Not Success yet!

In [9]:
## Only extract one row! But how to get all rows?
table1=wait.until(EC.visibility_of_element_located((By.XPATH,'//*[contains(@class,"data-table fixed")]//tbody//tr')))
data1 = [item.text for item in table1.find_elements_by_xpath(".//*[self::td or self::th]")]
table2=wait.until(EC.visibility_of_element_located((By.XPATH,'//*[contains(@class,"data-table main")]//tbody//tr')))
data2 = [item.text for item in table2.find_elements_by_xpath(".//*[self::td or self::th]")]
row_values=data1+data2
print(row_values)

['1', '虚空之女', 'ADC', '54.5 % ，162 场', '40.7 %，121 场', '58.0 %，94 场', '4.4 KDA\n5.2 / 2.2 / 4.4']
